In [1]:
import os
import openai
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from llama_index import LangchainEmbedding
from llama_index import download_loader
from llama_index import (
    GPTVectorStoreIndex,
    SimpleDirectoryReader, 
    LLMPredictor,
    ServiceContext,
    StorageContext,
)
from dotenv import load_dotenv

# Chargement des variables d'environnement depuis le fichier .env
load_dotenv('./.env')

True

Configuration de l'API OpenAI

In [2]:
# Configuration de l'API OpenAI
openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"
openai.api_base = "https://innoopenai.openai.azure.com/"
openai.api_key = os.getenv('OPENAI_API_KEY')

In [3]:
#set context window
context_window = 4096

#set number of output tokens
num_output = 512

In [4]:
# Initialisation de l'objet AzureOpenAI
# test1 représente le nom de déployment model sur Azure (le nom du modèle gpt35turbo)
llm = AzureChatOpenAI(deployment_name="test1", temperature=0.1, max_tokens=num_output, openai_api_version=openai.api_version, model_kwargs={
    "api_key": openai.api_key,
    "api_base": openai.api_base,
    "api_type": openai.api_type,
    "api_version": openai.api_version,
})
llm_predictor = LLMPredictor(llm=llm)

# Initialisation de l'objet LangchainEmbedding pour l'indexation des documents à partir ici du modèle ada-002 nommé ada-test dans Azureembedding_llm = LangchainEmbedding(
embedding_llm = LangchainEmbedding(
    OpenAIEmbeddings(
        model="text-embedding-ada-002",
        deployment="ada-test",
        openai_api_key= openai.api_key,
        openai_api_base=openai.api_base,
        openai_api_type=openai.api_type,
        openai_api_version=openai.api_version,
    ),
    embed_batch_size=1,
)

pdf, txt

In [5]:
# Chargement des documents à partir du répertoire './data'
documents = SimpleDirectoryReader('./data').load_data()
#permet de vérifier que les docs ont bien étés chargés
print('Document ID:', documents[0].doc_id)
print((f"Loaded doc with {len(documents)} pages"))

Document ID: 8c6e9d0c-7ec9-4707-9783-8ea56d2b3b22
Loaded doc with 33 pages


Load qu'une seule page d'un site

In [ ]:
BeautifulSoupWebReader = download_loader("BeautifulSoupWebReader")
loader = BeautifulSoupWebReader()
documents = loader.load_data(urls=['https://www.equans.fr/'])
#permet de vérifier que les docs ont bien étés chargés
print('Document ID:', documents[0].doc_id)

In [6]:
service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor,
    embed_model=embedding_llm,
    context_window=context_window,
    num_output=num_output,
)

In [7]:
# Création de l'index à partir des documents et du service_context
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)
index.storage_context.persist(persist_dir="./storage")
print((f"Finished building doc n°1 index with {len(index.docstore.docs)} nodes"))

Finished building doc n°1 index with 41 nodes


In [ ]:
# Chargement de l'index à partir du stockage
from llama_index import load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir="./storage")
index = load_index_from_storage(storage_context, service_context=service_context)
print((f"Finished loading doc n°1 index from storage with {len(index.docstore.docs)} nodes"))

In [8]:
from llama_index import Prompt
template = (
    "Tu trouveras ci-dessous des informations contextuelles. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Tu es un assistant technique de la socitété Equans via un chatbot. Tu donnes une assistance technique aux questions que te poseras l'utilisateur."
    "D'après le contexte, réponds à la question en français uniquement, même si la question est posée dans une autre langue. Réponds donc à la question:{query_str}\n"
    "Il se peut que l'utilisateur te pose des questions sur des parties spécifiques du document. Essaye de les retrouver et de répondre à la question"
    "Si la question n'a rien à voir avec les documents, réponds simplement : 'Je suis désolé, je n'ai pas pu trouver la réponse dans les documents que vous m'avez donné.' sauf si on te demande des questions sur toi(ex: Bonjour ou qui es-tu ?)."
)
qa_template = Prompt(template)

In [11]:
# Requête envoyée au modèle
context = "ce document est un rapport sur les systèmes de repérage et de détresse"
query = "de quoi parle ce document ?"
query_engine = index.as_query_engine(similarity_top_k=3, text_qa_template=qa_template)
answer = query_engine.query(query)

# Affichage des résultats
#print(answer.get_formatted_sources())
print('Utilisateur:', query)
print('EquansGPT:', answer)

Utilisateur: de quoi parle ce document ?
EquansGPT: Ce document parle des ondes électromagnétiques et de leurs applications dans différents systèmes tels que la sécurité, la détection d'objets en mouvement, la cartographie et la localisation d'objets dans des environnements à faible visibilité. Il explique également le phénomène d'écho utilisé dans la détection d'objets à l'aide de faisceaux lumineux.


In [12]:
for node in answer.source_nodes:
    print('-----')
    text_fmt = node.node.text.strip().replace('\n', ' ')[:1000]
    print(f"Text:\t {text_fmt} ...")
    print(f'Metadata:\t {node.node.extra_info}')
    print(f'Score:\t {node.score:.3f}')

-----
Text:	 une  meilleure compréhension et à une utilisation optimisée de ces systèm es pour la sécurité et le  bien-être de tous. ...
Metadata:	 {'page_label': '4', 'file_name': 'Rapport bibliographique AZOULAY_BERI_DELANGLE.pdf'}
Score:	 0.829
-----
Text:	 maladies.   De l’autre côté, les ondes au -dessus de la lumière visible (infrarouges, micro -ondes et  radios) ont de nombreuses applications dans les systèmes de repérages et de détresse. On les  utilise notamment dans la transmission d’informations, la détection d’objets en mouvements,  dans la cartographie, la localisation d’objets dans des environnements à faible visibilité. ...
Metadata:	 {'page_label': '9', 'file_name': 'Rapport bibliographique AZOULAY_BERI_DELANGLE.pdf'}
Score:	 0.810
-----
Text:	 de faisceaux par seconde sur une surface qui les réfléchit  et revient à sa s ource. On mesure ensuite le temps que met la lumière à revenir vers lui. C’est  ce qu’on appelle le phénomène d’écho. Plus l’écho est fort, plus l’obje